# Skip-gram Word2Vec

In this notebook, I'll lead you through using PyTorch to implement the [Word2Vec algorithm](https://en.wikipedia.org/wiki/Word2vec) using the skip-gram architecture. By implementing this, you'll learn about embedding words for use in natural language processing. This will come in handy when dealing with things like machine translation.

## Readings

Here are the resources I used to build this notebook. I suggest reading these either beforehand or while you're working on this material.

* A really good [conceptual overview](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/) of Word2Vec from Chris McCormick 
* [First Word2Vec paper](https://arxiv.org/pdf/1301.3781.pdf) from Mikolov et al.
* [Neural Information Processing Systems, paper](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf) with improvements for Word2Vec also from Mikolov et al.

---
## Word embeddings

When you're dealing with words in text, you end up with tens of thousands of word classes to analyze; one for each word in a vocabulary. Trying to one-hot encode these words is massively inefficient because most values in a one-hot vector will be set to zero. So, the matrix multiplication that happens in between a one-hot input vector and a first, hidden layer will result in mostly zero-valued hidden outputs.

To solve this problem and greatly increase the efficiency of our networks, we use what are called **embeddings**. Embeddings are just a fully connected layer like you've seen before. We call this layer the embedding layer and the weights are embedding weights. We skip the multiplication into the embedding layer by instead directly grabbing the hidden layer values from the weight matrix. We can do this because the multiplication of a one-hot encoded vector with a matrix returns the row of the matrix corresponding the index of the "on" input unit.

<img src='assets/lookup_matrix.png' width=50%>

Instead of doing the matrix multiplication, we use the weight matrix as a lookup table. We encode the words as integers, for example "heart" is encoded as 958, "mind" as 18094. Then to get hidden layer values for "heart", you just take the 958th row of the embedding matrix. This process is called an **embedding lookup** and the number of hidden units is the **embedding dimension**.
 
There is nothing magical going on here. The embedding lookup table is just a weight matrix. The embedding layer is just a hidden layer. The lookup is just a shortcut for the matrix multiplication. The lookup table is trained just like any weight matrix.

Embeddings aren't only used for words of course. You can use them for any model where you have a massive number of classes. A particular type of model called **Word2Vec** uses the embedding layer to find vector representations of words that contain semantic meaning.

---
## Word2Vec

The Word2Vec algorithm finds much more efficient representations by finding vectors that represent the words. These vectors also contain semantic information about the words.

<img src="assets/context_drink.png" width=40%>

Words that show up in similar **contexts**, such as "coffee", "tea", and "water" will have vectors near each other. Different words will be further away from one another, and relationships can be represented by distance in vector space.


There are two architectures for implementing Word2Vec:
>* CBOW (Continuous Bag-Of-Words) and 
* Skip-gram

<img src="assets/word2vec_architectures.png" width=60%>

In this implementation, we'll be using the **skip-gram architecture** with **negative sampling** because it performs better than CBOW and trains faster with negative sampling. Here, we pass in a word and try to predict the words surrounding it in the text. In this way, we can train the network to learn representations for words that show up in similar contexts.

---
## Loading Data

Next, we'll ask you to load in data and place it in the `data` directory

1. Load the [text8 dataset](https://s3.amazonaws.com/video.udacity-data.com/topher/2018/October/5bbe6499_text8/text8.zip); a file of cleaned up *Wikipedia article text* from Matt Mahoney. 
2. Place that data in the `data` folder in the home directory.
3. Then you can extract it and delete the archive, zip file to save storage space.

After following these steps, you should have one file in your data directory: `data/text8`.

In [1]:
# read in the extracted text file      
with open('data/text8') as f:
    text = f.read()

# print out the first 100 characters
print(text[:100])

 anarchism originated as a term of abuse first used against early working class radicals including t


## Pre-processing

Here I'm fixing up the text to make training easier. This comes from the `utils.py` file. The `preprocess` function does a few things:
>* It converts any punctuation into tokens, so a period is changed to ` <PERIOD> `. In this data set, there aren't any periods, but it will help in other NLP problems. 
* It removes all words that show up five or *fewer* times in the dataset. This will greatly reduce issues due to noise in the data and improve the quality of the vector representations. 
* It returns a list of words in the text.

This may take a few seconds to run, since our text file is quite large. If you want to write your own functions for this stuff, go for it!

In [2]:
import utils

# get list of words
words = utils.preprocess(text)
print(words[:30])

['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including', 'the', 'diggers', 'of', 'the', 'english', 'revolution', 'and', 'the', 'sans', 'culottes', 'of', 'the', 'french', 'revolution', 'whilst']


In [3]:
# print some stats about this word data
print("Total words in text: {}".format(len(words)))
print("Unique words: {}".format(len(set(words)))) # `set` removes any duplicate words

Total words in text: 16680599
Unique words: 63641


### Dictionaries

Next, I'm creating two dictionaries to convert words to integers and back again (integers to words). This is again done with a function in the `utils.py` file. `create_lookup_tables` takes in a list of words in a text and returns two dictionaries.
>* The integers are assigned in descending frequency order, so the most frequent word ("the") is given the integer 0 and the next most frequent is 1, and so on. 

Once we have our dictionaries, the words are converted to integers and stored in the list `int_words`.

In [4]:
vocab_to_int, int_to_vocab = utils.create_lookup_tables(words)
int_words = [vocab_to_int[word] for word in words]

print(int_words[:30])

[5233, 3080, 11, 5, 194, 1, 3133, 45, 58, 155, 127, 741, 476, 10571, 133, 0, 27349, 1, 0, 102, 854, 2, 0, 15067, 58112, 1, 0, 150, 854, 3580]


## Subsampling

Words that show up often such as "the", "of", and "for" don't provide much context to the nearby words. If we discard some of them, we can remove some of the noise from our data and in return get faster training and better representations. This process is called subsampling by Mikolov. For each word $w_i$ in the training set, we'll discard it with probability given by 

$$ P(w_i) = 1 - \sqrt{\frac{t}{f(w_i)}} $$

where $t$ is a threshold parameter and $f(w_i)$ is the frequency of word $w_i$ in the total dataset.

> Implement subsampling for the words in `int_words`. That is, go through `int_words` and discard each word given the probablility $P(w_i)$ shown above. Note that $P(w_i)$ is the probability that a word is discarded. Assign the subsampled data to `train_words`.

In [15]:
from collections import Counter
import random
import numpy as np

threshold = 1e-5
word_counts = Counter(int_words)
print(list(word_counts.items())[0])  # dictionary of int_words, how many times they appear

# discard some frequent words, according to the subsampling equation
# create a new list of words for training
total_count = len(int_words)
freqs = {word: count/total_count for word, count in word_counts.items()}
probabilities = 1 - np.sqrt(threshold / (np.array(list(word_counts.values())) / total_count))
probability_lookup = { word: prob for word, prob in zip(word_counts.keys(), probabilities) }
train_words = [word for word in int_words if random.random() > probability_lookup[word]]

print("Before: ", len(int_words))
print("After:", len(train_words))
print(train_words[:30])

(5233, 303)
Before:  16680599
After: 4628989
[5233, 3133, 58, 741, 10571, 27349, 15067, 58112, 3580, 190, 10712, 3672, 7088, 5233, 44611, 2877, 186, 5233, 2621, 8983, 279, 4147, 141, 1137, 4860, 6753, 7573, 247, 11064, 51]


## Making batches

Now that our data is in good shape, we need to get it into the proper form to pass it into our network. With the skip-gram architecture, for each word in the text, we want to define a surrounding _context_ and grab all the words in a window around that word, with size $C$. 

From [Mikolov et al.](https://arxiv.org/pdf/1301.3781.pdf): 

"Since the more distant words are usually less related to the current word than those close to it, we give less weight to the distant words by sampling less from those words in our training examples... If we choose $C = 5$, for each training word we will select randomly a number $R$ in range $[ 1: C ]$, and then use $R$ words from history and $R$ words from the future of the current word as correct labels."

> **Exercise:** Implement a function `get_target` that receives a list of words, an index, and a window size, then returns a list of words in the window around the index. Make sure to use the algorithm described above, where you chose a random number of words to from the window.

Say, we have an input and we're interested in the idx=2 token, `741`: 
```
[5233, 58, 741, 10571, 27349, 0, 15067, 58112, 3580, 58, 10712]
```

For `R=2`, `get_target` should return a list of four values:
```
[5233, 58, 10571, 27349]
```

In [6]:
def get_target(words, idx, window_size=5):
    ''' Get a list of words in a window around an index. '''

    word_range = random.randint(1, window_size)
    
    return words[max(0, idx - word_range):idx] + words[idx + 1:min(len(words), idx + 1 + word_range)]

In [7]:
# test your code!

# run this cell multiple times to check for random window selection
int_text = [i for i in range(10)]
print('Input: ', int_text)
idx=5 # word index of interest

target = get_target(int_text, idx=idx, window_size=5)
print('Target: ', target)  # you should get some indices around the idx

Input:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Target:  [3, 4, 6, 7]


### Generating Batches 

Here's a generator function that returns batches of input and target data for our model, using the `get_target` function from above. The idea is that it grabs `batch_size` words from a words list. Then for each of those batches, it gets the target words in a window.

In [8]:
def get_batches(words, batch_size, window_size=5):
    ''' Create a generator of word batches as a tuple (inputs, targets) '''
    
    n_batches = len(words)//batch_size
    
    # only full batches
    words = words[:n_batches*batch_size]
    
    for idx in range(0, len(words), batch_size):
        x, y = [], []
        batch = words[idx:idx+batch_size]
        for ii in range(len(batch)):
            batch_x = batch[ii]
            batch_y = get_target(batch, ii, window_size)
            y.extend(batch_y)
            x.extend([batch_x]*len(batch_y))
        yield x, y
    

In [9]:
int_text = [i for i in range(20)]
x,y = next(get_batches(int_text, batch_size=4, window_size=5))

print('x\n', x)
print('y\n', y)

x
 [0, 0, 0, 1, 1, 1, 2, 2, 3, 3, 3]
y
 [1, 2, 3, 0, 2, 3, 1, 3, 0, 1, 2]


---
## Validation

Here, I'm creating a function that will help us observe our model as it learns. We're going to choose a few common words and few uncommon words. Then, we'll print out the closest words to them using the cosine similarity: 

<img src="assets/two_vectors.png" width=30%>

$$
\mathrm{similarity} = \cos(\theta) = \frac{\vec{a} \cdot \vec{b}}{|\vec{a}||\vec{b}|}
$$


We can encode the validation words as vectors $\vec{a}$ using the embedding table, then calculate the similarity with each word vector $\vec{b}$ in the embedding table. With the similarities, we can print out the validation words and words in our embedding table semantically similar to those words. It's a nice way to check that our embedding table is grouping together words with similar semantic meanings.

In [10]:
def cosine_similarity(embedding, valid_size=16, valid_window=100, device='cpu'):
    """ Returns the cosine similarity of validation words with words in the embedding matrix.
        Here, embedding should be a PyTorch embedding module.
    """
    
    # Here we're calculating the cosine similarity between some random words and 
    # our embedding vectors. With the similarities, we can look at what words are
    # close to our random words.
    
    # sim = (a . b) / |a||b|
    
    embed_vectors = embedding.weight
    
    # magnitude of embedding vectors, |b|
    magnitudes = embed_vectors.pow(2).sum(dim=1).sqrt().unsqueeze(0)
    
    # pick N words from our ranges (0,window) and (1000,1000+window). lower id implies more frequent 
    valid_examples = np.array(random.sample(range(valid_window), valid_size//2))
    valid_examples = np.append(valid_examples,
                               random.sample(range(1000,1000+valid_window), valid_size//2))
    valid_examples = torch.LongTensor(valid_examples).to(device)
    
    valid_vectors = embedding(valid_examples)
    similarities = torch.mm(valid_vectors, embed_vectors.t())/magnitudes
        
    return valid_examples, similarities

---
# SkipGram model

Define and train the SkipGram model. 
> You'll need to define an [embedding layer](https://pytorch.org/docs/stable/nn.html#embedding) and a final, softmax output layer.

An Embedding layer takes in a number of inputs, importantly:
* **num_embeddings** – the size of the dictionary of embeddings, or how many rows you'll want in the embedding weight matrix
* **embedding_dim** – the size of each embedding vector; the embedding dimension

Below is an approximate diagram of the general structure of our network.
<img src="assets/skip_gram_arch.png" width=60%>

>* The input words are passed in as batches of input word tokens. 
* This will go into a hidden layer of linear units (our embedding layer). 
* Then, finally into a softmax output layer. 

We'll use the softmax layer to make a prediction about the context words by sampling, as usual.

---
## Negative Sampling

For every example we give the network, we train it using the output from the softmax layer. That means for each input, we're making very small changes to millions of weights even though we only have one true example. This makes training the network very inefficient. We can approximate the loss from the softmax layer by only updating a small subset of all the weights at once. We'll update the weights for the correct example, but only a small number of incorrect, or noise, examples. This is called ["negative sampling"](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf). 

There are two modifications we need to make. First, since we're not taking the softmax output over all the words, we're really only concerned with one output word at a time. Similar to how we use an embedding table to map the input word to the hidden layer, we can now use another embedding table to map the hidden layer to the output word. Now we have two embedding layers, one for input words and one for output words. Secondly, we use a modified loss function where we only care about the true example and a small subset of noise examples.

$$
- \large \log{\sigma\left(u_{w_O}\hspace{0.001em}^\top v_{w_I}\right)} -
\sum_i^N \mathbb{E}_{w_i \sim P_n(w)}\log{\sigma\left(-u_{w_i}\hspace{0.001em}^\top v_{w_I}\right)}
$$

This is a little complicated so I'll go through it bit by bit. $u_{w_O}\hspace{0.001em}^\top$ is the embedding vector for our "output" target word (transposed, that's the $^\top$ symbol) and $v_{w_I}$ is the embedding vector for the "input" word. Then the first term 

$$\large \log{\sigma\left(u_{w_O}\hspace{0.001em}^\top v_{w_I}\right)}$$

says we take the log-sigmoid of the inner product of the output word vector and the input word vector. Now the second term, let's first look at 

$$\large \sum_i^N \mathbb{E}_{w_i \sim P_n(w)}$$ 

This means we're going to take a sum over words $w_i$ drawn from a noise distribution $w_i \sim P_n(w)$. The noise distribution is basically our vocabulary of words that aren't in the context of our input word. In effect, we can randomly sample words from our vocabulary to get these words. $P_n(w)$ is an arbitrary probability distribution though, which means we get to decide how to weight the words that we're sampling. This could be a uniform distribution, where we sample all words with equal probability. Or it could be according to the frequency that each word shows up in our text corpus, the unigram distribution $U(w)$. The authors found the best distribution to be $U(w)^{3/4}$, empirically. 

Finally, in 

$$\large \log{\sigma\left(-u_{w_i}\hspace{0.001em}^\top v_{w_I}\right)},$$ 

we take the log-sigmoid of the negated inner product of a noise vector with the input vector. 

<img src="assets/neg_sampling_loss.png" width=50%>

To give you an intuition for what we're doing here, remember that the sigmoid function returns a probability between 0 and 1. The first term in the loss pushes the probability that our network will predict the correct word $w_O$ towards 1. In the second term, since we are negating the sigmoid input, we're pushing the probabilities of the noise words towards 0.

In [11]:
import torch
from torch import nn
import torch.optim as optim

In [12]:
class SkipGramNeg(nn.Module):
    def __init__(self, n_vocab, n_embed, noise_dist=None):
        super().__init__()
        
        self.n_vocab = n_vocab
        self.n_embed = n_embed
        self.noise_dist = noise_dist
        
        # define embedding layers for input and output words
        self.in_embed = nn.Embedding(n_vocab, n_embed)
        self.out_embed = nn.Embedding(n_vocab, n_embed)
        
        # Initialize both embedding tables with uniform distribution
        # I believe this helps with convergence
        self.in_embed.weight.data.uniform_(-1, 1)
        self.out_embed.weight.data.uniform_(-1, 1)
        
    def forward_input(self, input_words):
        # return input vector embeddings
        return self.in_embed(input_words)
    
    def forward_output(self, output_words):
        # return output vector embeddings
        return self.out_embed(output_words)
    
    def forward_noise(self, batch_size, n_samples):
        """ Generate noise vectors with shape (batch_size, n_samples, n_embed)"""
        if self.noise_dist is None:
            # Sample words uniformly
            noise_dist = torch.ones(self.n_vocab)
        else:
            noise_dist = self.noise_dist
            
        # Sample words from our noise distribution
        noise_words = torch.multinomial(noise_dist,
                                        batch_size * n_samples,
                                        replacement=True)
        
        device = "cuda" if model.out_embed.weight.is_cuda else "cpu"
        noise_words = noise_words.to(device)
        
        ## get the noise embeddings
        # reshape the embeddings so that they have dims (batch_size, n_samples, n_embed)
        return self.out_embed(noise_words).view(batch_size, n_samples, self.n_embed)

In [13]:
class NegativeSamplingLoss(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, input_vectors, output_vectors, noise_vectors):
        
        batch_size, embed_size = input_vectors.shape
        
        # Input vectors should be a batch of column vectors
        input_vectors = input_vectors.view(batch_size, embed_size, 1)
        
        # Output vectors should be a batch of row vectors
        output_vectors = output_vectors.view(batch_size, 1, embed_size)
        
        # bmm = batch matrix multiplication
        # correct log-sigmoid loss
        out_loss = torch.bmm(output_vectors, input_vectors).sigmoid().log()
        out_loss = out_loss.squeeze()
        
        # incorrect log-sigmoid loss
        noise_loss = torch.bmm(noise_vectors.neg(), input_vectors).sigmoid().log()
        noise_loss = noise_loss.squeeze().sum(1)  # sum the losses over the sample of noise vectors

        # negate and sum correct and noisy log-sigmoid losses
        # return average batch loss
        return -(out_loss + noise_loss).mean()

### Training

Below is our training loop, and I recommend that you train on GPU, if available.

In [16]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Get our noise distribution
# Using word frequencies calculated earlier in the notebook
word_freqs = np.array(sorted(freqs.values(), reverse=True))
unigram_dist = word_freqs/word_freqs.sum()
noise_dist = torch.from_numpy(unigram_dist**(0.75)/np.sum(unigram_dist**(0.75)))

# instantiating the model
embedding_dim = 300
model = SkipGramNeg(len(vocab_to_int), embedding_dim, noise_dist=noise_dist).to(device)

# using the loss that we defined
criterion = NegativeSamplingLoss() 
optimizer = optim.Adam(model.parameters(), lr=0.003)

print_every = 1500
steps = 0
epochs = 5

# train for some number of epochs
for e in range(epochs):
    
    # get our input, target batches
    for input_words, target_words in get_batches(train_words, 32):
        steps += 1
        inputs, targets = torch.LongTensor(input_words), torch.LongTensor(target_words)
        inputs, targets = inputs.to(device), targets.to(device)
        
        # input, outpt, and noise vectors
        input_vectors = model.forward_input(inputs)
        output_vectors = model.forward_output(targets)
        noise_vectors = model.forward_noise(inputs.shape[0], 5)

        # negative sampling loss
        loss = criterion(input_vectors, output_vectors, noise_vectors)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # loss stats
        if steps % print_every == 0:
            print("Epoch: {}/{}".format(e+1, epochs))
            print("Loss: ", loss.item()) # avg batch loss at this point in training
            valid_examples, valid_similarities = cosine_similarity(model.in_embed, device=device)
            _, closest_idxs = valid_similarities.topk(6)

            valid_examples, closest_idxs = valid_examples.to('cpu'), closest_idxs.to('cpu')
            for ii, valid_idx in enumerate(valid_examples):
                closest_words = [int_to_vocab[idx.item()] for idx in closest_idxs[ii]][1:]
                print(int_to_vocab[valid_idx.item()] + " | " + ', '.join(closest_words))
            print("...\n")

Epoch: 1/5
Loss:  13.300207138061523
if | iulia, cryptanalysis, arching, informational, gj
have | panzers, gamespot, erasmus, commandments, naturally
and | belgique, experiential, nausica, code, windmill
after | noonan, mirror, sieves, predominate, kruspe
war | delos, constitutionally, chalons, pic, triumphal
they | fredrik, carabiners, liebert, impassable, remini
between | bikers, eks, pronounced, refers, miners
of | romanisation, patriarchate, low, cori, objections
cost | warnings, reproduces, jona, guidebook, jeffersonian
orthodox | integrator, salomon, greenfacts, sloppy, madelyne
universe | startled, congreve, supranationalism, auric, shining
behind | striped, mag, emmanuelle, tengri, hermannus
powers | injunctions, amn, pulsed, underway, signage
question | ethiopian, portmanteaus, nearer, signed, tournament
event | ardant, artworks, endemol, pod, weissmuller
troops | familycolor, brotherly, usefulness, marceau, drove
...

Epoch: 1/5
Loss:  10.79105281829834
people | juggalos, ed,

Epoch: 1/5
Loss:  5.484482765197754
history | mcelroy, atheism, newspaper, corp, comedian
d | eight, six, one, four, french
b | eight, parity, seven, nine, writer
six | seven, nine, d, three, win
war | ingold, ringing, cornerstone, tomb, learnt
would | hawai, calculator, archimedes, midian, arafat
years | person, agreements, bipedal, greenwich, elk
where | derby, asparagales, theorems, motorway, melanippe
hold | logie, ifv, task, airport, caloric
know | classified, webmineral, owes, stays, launched
event | refrained, beginning, judge, centrality, void
bill | akh, otto, backside, elvish, danish
animals | iud, supervisor, facial, rykodisc, malmsteen
universe | golgi, startled, enumerative, xxiii, shining
applied | mohawk, hymns, action, stosunku, rampart
heavy | pilate, landholding, expenditures, hitler, australis
...

Epoch: 1/5
Loss:  4.977888584136963
use | curae, squads, vanderbilt, ethereal, imbued
or | have, mariculture, to, is, agreed
than | journalists, enhanced, also, untimely, 

Epoch: 1/5
Loss:  5.392176151275635
on | blue, simulation, dendrites, three, axles
that | cases, hands, in, is, vs
years | hoffmann, gaku, specifically, proportionally, their
are | devoutly, harmless, length, into, offshoot
only | soprano, concerto, bypassed, significantly, of
seven | zero, companionate, john, film, isbn
when | sandow, have, ecuador, valence, assume
i | bysshe, burley, reimbursement, jurors, boxers
recorded | debunking, quarreled, maneuvers, electric, elmira
applied | be, shamil, kishore, bouncer, banat
something | hindering, slacks, vagantes, nemo, ascertained
hold | ripping, sierpinski, fillings, scapa, demyelinating
question | devin, lessons, wakefulness, oceanian, terminates
creation | saarland, bean, accompli, bek, beaver
police | along, problematic, turbofans, cambria, antonym
ocean | sheffield, twisted, fifty, mebyon, west
...

Epoch: 1/5
Loss:  5.024831295013428
can | brain, ability, each, number, produce
between | receive, quantization, applesoft, financial, o

Epoch: 1/5
Loss:  4.395859241485596
the | and, of, zero, with, or
not | concerning, that, drugs, although, cannot
had | adopted, grew, offices, synergistic, he
other | with, caused, about, swelling, statistically
first | five, official, kr, abusive, aesop
on | when, kelsey, sportsman, of, into
with | the, light, to, easily, into
its | those, in, both, with, uncles
articles | sentinel, necessitated, sammet, scholars, proteles
discovered | managed, aid, around, changing, scorned
applied | mika, spreadsheet, nomen, demonstrated, usage
accepted | distorted, transference, ideological, unified, insubordination
mathematics | theorem, if, theory, omphalos, g
bbc | radio, durkheim, www, canadian, projects
derived | word, thoughts, frans, can, anthropologists
powers | indeed, indictment, pavilion, discordian, misinterpretation
...

Epoch: 1/5
Loss:  3.7877464294433594
world | united, arts, capture, gorge, goldsworthy
people | from, community, whether, next, of
years | university, expending, live

Epoch: 1/5
Loss:  3.8311712741851807
were | mask, have, support, actions, practices
however | being, from, work, practice, good
zero | two, four, five, nine, one
they | did, during, claims, except, turgot
this | because, explicitly, systems, alternative, and
and | the, to, part, of, in
over | independent, ru, commercial, oil, gold
known | the, intensified, was, other, backpack
universe | primordial, originally, fictional, vision, matter
additional | manner, sequence, modern, random, scroll
joseph | nobel, harold, born, jan, tommy
paris | oscars, freescale, sobel, mbe, prefixing
assembly | government, elections, provincial, foreign, court
brother | corrie, anniversary, song, his, series
heavy | machines, surfaces, mongi, isolated, explosions
channel | subculture, mhz, columbus, five, satellite
...

Epoch: 1/5
Loss:  4.216150283813477
b | z, c, h, y, x
were | old, last, typically, lived, confederate
while | to, different, because, there, confusion
that | what, these, criticism, avoid, po

...

Epoch: 5/5
Loss:  4.217324256896973
who | that, cromwellian, were, fabrizio, a
six | zero, four, eight, five, c
a | to, of, the, with, property
of | the, seven, a, one, in
s | nine, tribute, one, brother, first
be | integral, however, problem, formula, allows
is | defined, spectral, of, e, energy
if | suppose, then, must, impossible, be
test | surgical, dive, certification, risk, tests
mathematics | mathematicians, theorems, equations, properties, mathematical
numerous | assyria, zealots, khedive, s, please
marriage | niece, sisters, elector, boleyn, luther
paris | france, lyon, palace, sur, ville
channel | tv, cable, mbit, catv, mxy
joseph | louise, nicholas, horatio, walter, nine
ocean | kilometers, coastal, plateau, tributaries, subtropical
...

Epoch: 5/5
Loss:  3.9425337314605713
state | equations, democracy, relations, general, fructose
system | application, practical, computers, an, cpu
see | photon, distance, computation, applying, solar
not | these, are, by, or, become
only | 

Epoch: 5/5
Loss:  2.7496798038482666
after | not, death, antipopes, pius, bledsoe
at | seven, university, he, four, visited
has | the, though, to, of, only
zero | seven, two, four, five, eight
united | country, military, trade, airport, u
d | l, sident, physiologist, k, georgi
will | joram, order, take, purely, what
into | types, effect, derived, atoms, liberating
taking | finally, therapist, promotion, appropriate, boomerangs
hold | must, hypocrisy, their, within, righteousness
channel | cable, broadcast, shortwave, transmitters, broadband
joseph | impressed, d, alexander, composer, walter
magazine | games, cgi, sold, developer, comics
ocean | arctic, atlantic, intelsat, windward, nicobar
professional | women, amateur, profit, graduate, varsity
http | www, htm, com, org, portal
...

Epoch: 5/5
Loss:  4.310762882232666
after | bledsoe, bibulus, everlasting, gave, bologna
or | substituents, simply, not, the, cases
only | of, thus, evolve, has, using
however | instances, say, term, since

Epoch: 5/5
Loss:  4.405395984649658
used | different, derive, pronunciation, usage, languages
this | known, caste, offer, etiology, the
seven | two, six, three, eight, five
would | another, on, potential, regained, ever
is | not, meaning, often, non, kinds
where | into, field, then, hilbert, line
up | it, have, foot, daugava, sneaking
had | after, been, kitty, renamed, reestablishing
mainly | dispersed, beaker, bitterness, meo, bua
brother | mcduck, wife, daughter, elder, matilda
ocean | coast, nicobar, km, islands, atlantic
magazine | journals, publisher, interview, novels, animator
something | feeling, strange, feels, very, rude
versions | version, ported, mouse, proprietary, syndication
liberal | conservative, leaders, nationalist, liberalism, parties
bible | tanakh, biblical, ezra, esdras, nevi
...

Epoch: 5/5
Loss:  4.360540390014648
s | a, development, railway, demographically, during
system | integrated, systems, newer, interconnected, zero
is | if, to, single, howto, as
so | th

Epoch: 5/5
Loss:  5.03341817855835
no | t, script, label, like, you
this | still, widely, and, sometimes, doubt
war | surrender, military, panzer, forces, nazi
about | may, at, man, her, his
five | zero, one, nine, six, three
these | frequently, single, a, lacks, specific
on | from, the, nine, man, retrial
he | his, in, face, from, a
experience | cause, moral, questions, drugs, emotional
mean | horizon, climatology, probability, vector, graph
freedom | united, citizen, policy, school, prohibition
behind | stuntman, harris, alcs, closest, devdas
scale | inflation, slide, viaducts, cheddar, workforce
road | cars, biography, front, sumner, york
engine | engines, fuel, piston, turbines, thrust
ocean | atlantic, islands, coast, bordering, inland
...

Epoch: 5/5
Loss:  4.428940296173096
up | already, entire, gina, revenues, finnish
four | five, two, zero, july, links
these | are, a, increasingly, frequently, for
or | is, may, the, pythons, either
history | one, historian, heavily, eastern, h

Epoch: 5/5
Loss:  4.5938029289245605
two | four, zero, six, seven, three
only | number, some, is, substantial, of
up | did, side, having, entire, hamon
th | century, east, western, history, asia
years | est, july, birth, male, age
for | national, self, the, brazzaville, soon
three | one, zero, four, eight, seven
eight | six, four, nine, seven, three
hit | pitcher, hits, offseason, yankee, hitter
universe | cosmological, newton, hubble, theory, relativity
powers | assisting, subjugation, gains, entente, temporary
mean | sum, harmonic, graph, equivalently, variable
pressure | internal, water, armies, temperature, explode
woman | wife, girl, wear, women, parents
writers | novelists, journal, etonians, satire, artists
construction | harbors, corridor, residences, skyscrapers, railway
...

Epoch: 5/5
Loss:  3.4299685955047607
three | seven, nine, one, eight, zero
th | asia, history, east, roman, thereafter
while | struggle, allegation, relieve, coat, show
who | he, him, devote, debater, cou

In [17]:
torch.save(model, "negative_sampling.pth")

In [ ]:
model = torch.load("negative_sampling.pth")

## Visualizing the word vectors

Below we'll use T-SNE to visualize how our high-dimensional word vectors cluster together. T-SNE is used to project these vectors into two dimensions while preserving local stucture. Check out [this post from Christopher Olah](http://colah.github.io/posts/2014-10-Visualizing-MNIST/) to learn more about T-SNE and other ways to visualize high-dimensional data.

In [34]:
%matplotlib inline
%config InlineBackend.figure_format = 'png'

import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

In [19]:
# getting embeddings from the embedding layer of our model, by name
embeddings = model.in_embed.weight.to('cpu').data.numpy()

In [27]:
viz_words = 380
tsne = TSNE()
embed_tsne = tsne.fit_transform(embeddings[:viz_words, :])

C:\Users\Creator\anaconda3\envs\udacity\lib\site-packages\sklearn\manifold\_t_sne.py:780: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
C:\Users\Creator\anaconda3\envs\udacity\lib\site-packages\sklearn\manifold\_t_sne.py:790: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(


In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))
for idx in range(viz_words):
    plt.scatter(*embed_tsne[idx, :], color='steelblue')
    plt.annotate(int_to_vocab[idx], (embed_tsne[idx, 0], embed_tsne[idx, 1]), alpha=0.7)